In [ ]:
from nose.tools import assert_equal, assert_almost_equal
import numpy as np

## Problem 1

Write a function `moving_average` that calculates the moving average of an array with a user-provide window (i.e., number of values to average). If you have an array `x` of length `n` and a window of `w` the output array `y` should have size `n` with each value being the average of the current and the last `w-1` values.
```python
y[1] = (x[0] + x[1]) / 2
y[2] = (x[1] + x[2]) / 2
```

In [ ]:
def moving_average(x, w):
# BEGIN SOLUTION
    out = np.zeros(len(x))
    out[:w-1] = np.nan
    for i in range(w-1, len(x)):
        out[i] = np.mean(x[max(0, i-w+1):i+1])
    return out
# END SOLUTION

In [ ]:
x = np.arange(20, dtype=np.float64)

In [ ]:
# The following should evaluate to True
np.all(moving_average(x, 2)[1:] == np.array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5,
       10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5]))

In [ ]:
assert np.all(moving_average(x, 2)[1:] == np.array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5,
       10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5]))

## Problem 2

Now take the function you wrote in Problem 1, and modify its name to `gmoving_average` and its signature (i.e., arguments) so it can be decorated with `numba`. Compare the timings of the two functions

In [ ]:
# BEGIN SOLUTION
from numba import guvectorize

@guvectorize(['void(float64[:], intp, float64[:])'],
             '(n),()->(n)')
def gmoving_average(x, w, out):
    out[:w-1] = np.nan
    for i in range(w-1, len(x)):
        out[i] = np.mean(x[max(0, i-w-1):i+1])
# END SOLUTION

In [ ]:
x = np.arange(20000, dtype=np.float64)

In [ ]:
out = np.zeros(20000, dtype=np.float64)
_ = gmoving_average(x, 2, out)
assert out[1] == 0.5

In [ ]:
# Time your functions here

## Problem 3

Convolution is the process of modifying the value of an array by summing its neighbors and weighing them by a kernel. It finds many applications in image processing, e.g. for edge detection.
Below you will find a schematic explaining the algorithm

![convolution](conv.png)

Write a function that takes a 2-D array (image) as an input argument and returns a convolved image with the following kernel

```python
[0 -1 0
 -1 4 -1
 0 -1 0]
```

In [ ]:
def convolve(arr, kernel):
# BEGIN SOLUTION
    k = kernel.shape[0] // 2
    m, n = arr.shape
    out = np.zeros(arr.shape)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if i >= k and i < m-k and j >= k and j < n-k:
                out[i, j] = np.sum(arr[i-k:i+k+1, j-k:j+k+1] * kernel)
            else:
                out[i, j] = 0.0
    return out
# END SOLUTION

In [ ]:
from matplotlib import image
import matplotlib.pyplot as plt
img = image.imread("koala.jpg") / 255
img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
plt.imshow(img)

In [ ]:
kernel = np.array([[0, 1, 0], [-1, 4, -1], [0, -1, 0]])
cimg = convolve(img, kernel)

In [ ]:
plt.imshow(cimg)

Confirm your answer by using the `Scipy` function `convolve`

In [ ]:
from scipy.ndimage import convolve as sconvolve
simg = sconvolve(img, kernel, mode='constant')

In [ ]:
plt.imshow(simg)

Confirm your answer by using the `Scipy` function

In [ ]:
assert_almost_equal(cimg[2, 1], simg[2, 1])